In [ ]:
%load_ext autoreload


import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.insert(0,"../../")
import oslo_method_python as om
import copy
%autoreload 2


import matplotlib as mpl
#mpl.style.use('article')

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('text', usetex=True)
rc('errorbar', capsize=1.5) # Set error bar style


%matplotlib notebook


In [ ]:
# Import raw matrix into instance of om.Matrix() and plot it
fname_raw = "data/Fe56-alfna_no_el_no_si.m"
raw = om.Matrix()
raw.load(fname_raw)
raw.plot(title="raw", zscale="log")

In [ ]:
# Cut away diagonal
Ex1 = 0
Eg1 = 800
E1 = [Ex1, Eg1]
Ex2 = 7300
Eg2 = 7500
E2 = [Ex2, Eg2]
#raw_diagcut = copy.copy(raw)
#raw_diagcut.matrix = om.cut_diagonal(raw.matrix, raw.E0_array, raw.E1_array, E1, E2)
raw.cut_diagonal(E1, E2)
raw.plot(zscale="log")

In [ ]:
# Drop Ex larger than Sn, about 11500 keV
raw.cut_rect(axis=0, E_limits=[0, 11500])
raw.plot()

In [ ]:
# Rebin Eg axis to get more handleable sizes
print("raw.calibration() =", raw.calibration())

a_Eg_out = 20 # keV
Eg_array_out = om.E_array_from_calibration(raw.calibration()["a10"],
                                           a_Eg_out,
                                           E_max=raw.E1_array.max())

raw_matrix_rebinned = om.rebin_matrix(raw.matrix,
                                  E_array_in=raw.E1_array,
                                  E_array_out=Eg_array_out,
                                  rebin_axis=1)
raw_rebinned = om.Matrix(matrix=raw_matrix_rebinned,
                         E0_array=raw.E0_array,
                         E1_array=Eg_array_out)


print("raw_rebinned.calibration() =", raw_rebinned.calibration())

print("new midbin calibration Eg: a0 = {:f}, a1 = {:f}".format(
    Eg_array_out[0]+a_Eg_out/2, a_Eg_out))

raw_rebinned.plot()


In [ ]:
# Put it into an instance of om.MatrixAnalysis() to do unfolding and first generation method:
ma = om.MatrixAnalysis()
# ma.raw = raw
ma.raw = raw_rebinned
ma.raw.plot()

In [ ]:
# Unfold the matrix. 
# I haven't implemented a fast enough routine for response function interpolation yet,
# so we have to load response functions with the right calibration, made by MAMA, from file.
fname_resp_mat = "data/response_matrix_20keV.m"
fname_resp_dat = "data/resp.dat"

# Call the unfolding algorithm:
diag_cut = {"Ex1": 0, "Eg1": 800, "Ex2": 7300, "Eg2": 7500}
ma.unfold(fname_resp_dat=fname_resp_dat, fname_resp_mat=fname_resp_mat, diag_cut=diag_cut,
          fill_and_remove_negative=True)
# The result is stored in the Matrix() instance ma.unfolded. Plot it:
ma.unfolded.plot()

In [ ]:
# Run first generation method
ma.first_generation_method(Ex_max = 11500, dE_gamma = 500,
                           fill_and_remove_negative=True)
ma.firstgen.plot(zmin=1e-3)

In [ ]:
# Do error propagation in order to obtain an error matrix on the first generation spectrum
# Note that it requires an instance of MatrixAnalysis as input
N_ensemble = 30 #100
ep = om.ErrorPropagation(ma, folder="error_propagation_ensemble", random_seed=481516)
ep.generate_ensemble(N_ensemble_members=N_ensemble, randomness="poisson", purge_files=False)

In [ ]:
f, (axraw, axunf, axfg) = plt.subplots(1, 3, sharey=True)

# Fetch and plot the standard deviation of raw:
std_raw = ep.std_raw
cbar_raw = std_raw.plot(ax=axraw)
f.colorbar(cbar_raw, ax=axraw)

# Fetch and plot the standard deviation of unfolded:
std_unfolded = ep.std_unfolded
cbar_unf = std_unfolded.plot(ax=axunf)
f.colorbar(cbar_unf, ax=axunf)

# Fetch and plot the standard deviation of firstgen:
std_firstgen = ep.std_firstgen
cbar_fg = std_firstgen.plot(ax=axfg)
f.colorbar(cbar_fg, ax=axfg)


# Add axis labels and titles
axraw.set_title("(a) raw")
axunf.set_title("(b) unfolded")
axfg.set_title("(c) first generation")

axraw.set_ylabel(r"$E_x \,\,\mathrm{(keV)}$")

axraw.set_xlabel(r"$E_\gamma \,\,\mathrm{(keV)}$")
axunf.set_xlabel(r"$E_\gamma \,\,\mathrm{(keV)}$")
axfg.set_xlabel(r"$E_\gamma \,\,\mathrm{(keV)}$")



f.set_size_inches((10, 5))
plt.tight_layout(True)


plt.savefig("Fe56-matrices_std_raw_unf_fg.png")

plt.show()

# Fit rho and T

In [ ]:
bin_width_out = 300
Ex_min = 6600
Ex_max = 11300
Eg_min = 2100

ftol = 1e-5 # Fitting tolerance (in some unknown, relative units)
maxfev = None # maximum number of function evaluations

#rho, T = om.fit_rho_T(ma.firstgen, firstgen_std, bin_width_out,
fits = om.FitRhoT(ma.firstgen, std_firstgen, bin_width_out,
                      Ex_min, Ex_max, Eg_min,
                      method="Powell",
                      options={'disp':True, 'ftol':ftol, 'maxfev':maxfev}
                      #error_normalization="approximate"
                     )
fits.fit()
rho = fits.rho
T = fits.T


f, (axrho, axgsf) = plt.subplots(1,2)
rho.plot(ax=axrho, yscale="log")
#T.plot(ax=axT, yscale="log")
gsf = om.Vector(E_array=T.E_array, vector=om.div0(T.vector, T.E_array**3))
gsf.plot(ax=axgsf, yscale="log")

plt.tight_layout(True)
plt.show()


In [ ]:
# Play with alpha parameter
alpha = 0.0005

f, (axrho, axgsf) = plt.subplots(1,2)
rho.transform(alpha=alpha, inplace=False).plot(ax=axrho, yscale="log")
#T.plot(ax=axT, yscale="log")
gsf = om.Vector(E_array=T.E_array, vector=om.div0(T.vector, T.E_array**3))
gsf.transform(alpha=alpha, inplace=False).plot(ax=axgsf, yscale="log")

axrho.set_xlim([0, rho.E_array.max()+500])

plt.tight_layout(True)
plt.show()

In [ ]:
# Fit the whole error ensemble using their common error matrix; use spread as gauge for uncert
# We replace the firstgen matrix of "ma_curr" every iteration.
ma_curr = copy.deepcopy(ma)

N_ensemble_fit = 10
if N_ensemble_fit > N_ensemble:
    raise ValueError("Not enough ensemble members")
    
try:
    # Load ensemble members of rho and T fits
    rho_ens = np.loadtxt("error_propagation_ensemble/rho_ensemble_fits-Nens_{:d}-ftol_{:.0e}-Exmin_{:d}-Exmax_{:d}-bin_width_{:d}.txt".format(N_ensemble_fit, ftol, Ex_min, Ex_max, bin_width_out))
    T_ens = np.loadtxt("error_propagation_ensemble/T_ensemble_fits-Nens_{:d}-ftol_{:.0e}-Exmin_{:d}-Exmax_{:d}-bin_width_{:d}.txt".format(N_ensemble_fit, ftol, Ex_min, Ex_max, bin_width_out))

except:
    rho_ens = np.zeros((N_ensemble_fit, len(rho.vector)))
    T_ens = np.zeros((N_ensemble_fit, len(T.vector)))
    for i_ens in range(N_ensemble_fit):
        print("Fitting ensemble member {:d}".format(i_ens))
        ma_curr.firstgen.matrix = ep.firstgen_ensemble[i_ens, :, :]
        ma_curr.firstgen.matrix = om.fill_negative(ma_curr.firstgen.matrix, window_size=10)
        ma_curr.firstgen.matrix[ma_curr.firstgen.matrix < 0] = 0
        #ma.firstgen.plot()
        fits_curr = om.FitRhoT(ma_curr.firstgen, std_firstgen, bin_width_out,
                          Ex_min, Ex_max, Eg_min,
                          method="Powell",
                          options={'disp':True, 'ftol':ftol, 'maxfev':maxfev}
                         )
        fits_curr.fit()
        rho_curr= fits_curr.rho
        T_curr = fits_curr.T
        rho_ens[i_ens, :] = rho_curr.vector
        T_ens[i_ens, :] = T_curr.vector
    
    # Save ensemble members of rho and T fits

    np.savetxt("error_propagation_ensemble/rho_ensemble_fits-Nens_{:d}-ftol_{:.0e}-Exmin_{:d}-Exmax_{:d}-bin_width_{:d}.txt".format(N_ensemble_fit, ftol, Ex_min, Ex_max, bin_width_out),
               rho_ens)
    np.savetxt("error_propagation_ensemble/T_ensemble_fits-Nens_{:d}-ftol_{:.0e}-Exmin_{:d}-Exmax_{:d}-bin_width_{:d}.txt".format(N_ensemble_fit, ftol, Ex_min, Ex_max, bin_width_out),
               T_ens)

In [ ]:
# Plot with uncertainties, only the transformed versions
alpha=0.0005
A = 0.05
B = 2*np.pi*0.6

color="teal"

f, (axrho_transf, axgsf_transf) = plt.subplots(1, 2)

rho_std = om.Vector(vector=np.std(rho_ens, axis=0), E_array=rho.E_array)

axrho_transf.errorbar(rho.E_array, rho.transform(const=A, alpha=alpha).vector,
                      yerr=rho_std.transform(const=A, alpha=alpha).vector,
                      label=r"$\rho$, transformed", fmt="o", markersize=2, capsize=1,
                      linewidth=0.9, color=color)

# Load and add published, normalized data:
rho_published = np.loadtxt("data/nld_56Fe_FG05_NaI.txt", skiprows=2)
rho_published[:, 1] *= 1000
axrho_transf.errorbar(rho_published[:, 1], rho_published[:, 2], yerr=rho_published[:, 3],
                      fmt="<", markersize=2, color="crimson", linewidth=0.9,
                      label=r"$\rho$, Larsen $\textit{et\,\,al.}$")

#axrho_transf.set_title("rho, transformed")
axrho_transf.legend()
axrho_transf.set_yscale("log")

T_std = om.Vector(vector=np.std(T_ens, axis=0), E_array=T.E_array)

# Convert T to gsf
gsf = om.Vector(E_array=T.E_array, vector=om.div0(T.vector, 2*np.pi*T.E_array**3))
gsf_std = om.Vector(E_array=T_std.E_array, vector=om.div0(T_std.vector, 2*np.pi*T_std.E_array**3))



axgsf_transf.errorbar(gsf.E_array, gsf.transform(const=B, alpha=alpha).vector,
                      yerr=gsf_std.transform(const=B, alpha=alpha).vector,
                      label=r"$\gamma$SF, transformed", linewidth=0.9,
                      fmt="o", markersize=2, capsize=1, color=color)

# Load and add published, normalized data:
gsf_published = np.loadtxt("data/gsf_56Fe_FG05_NaI.txt", skiprows=2)
gsf_published[:, 1] *= 1000
axgsf_transf.errorbar(gsf_published[:, 1], gsf_published[:, 2], yerr=gsf_published[:, 3],
                      fmt="<", markersize=2, color="crimson", linewidth=0.9,
                      label=r"$\gamma$SF, Larsen $\textit{et\,\,al.}$")

#axgsf_transf.set_title("gsf, transformed")
axgsf_transf.legend()
axgsf_transf.set_yscale("log")

# Axis labels
axrho_transf.set_title("(a) Level density")
axgsf_transf.set_title(r"(b) $\gamma$SF")

axrho_transf.set_ylabel(r"$\rho \,\, (\mathrm{MeV}^{-1})$")
axgsf_transf.set_ylabel(r"$f_1 \,\, (\mathrm{MeV}^{-3})$")

axrho_transf.set_xlabel(r"$E_x (\mathrm{MeV})$")
axgsf_transf.set_xlabel(r"$E_\gamma (\mathrm{MeV})$")

plt.tight_layout(True)

f.savefig("Dy164_rho_and_T_fit_alpha_transform.pdf")

plt.show()

In [ ]:
# Plot with PERCENTILE uncertainties, only the transformed versions
alpha=0.0007
A = 0.000009
B = 2*np.pi*9

color="lightblue"

f, (axrho_transf, axgsf_transf) = plt.subplots(2, 1)

rho_lower = om.Vector(vector=(- np.percentile(rho_ens, q=15.9, axis=0) + np.percentile(rho_ens, q=50, axis=0)),
                              E_array=rho.E_array)
rho_upper = om.Vector(vector=(np.percentile(rho_ens, q=84.1, axis=0) - np.percentile(rho_ens, q=50, axis=0)),
                              E_array=rho.E_array)

axrho_transf.errorbar(rho.E_array, rho.transform(const=A, alpha=alpha).vector,
                      yerr=(rho_lower.transform(const=A, alpha=alpha).vector,
                            rho_upper.transform(const=A, alpha=alpha).vector),
                      label=r"$\rho$, transformed", fmt="o", markersize=2, capsize=1,
                      linewidth=0.9, color=color)

# Load and add published, normalized data:
rho_published = np.loadtxt("data/nld_56Fe_FG05_NaI.txt", skiprows=2)
rho_published[:, 1] *= 1000
axrho_transf.errorbar(rho_published[:, 1], rho_published[:, 2], yerr=rho_published[:, 3],
                      fmt="<", markersize=2, color="seagreen", linewidth=0.9,
                      label=r"$\rho$, Larsen $\textit{et\,\,al.}$")

#axrho_transf.set_title("rho, transformed")
axrho_transf.legend()
axrho_transf.set_yscale("log")

T_lower = om.Vector(vector=(- np.percentile(T_ens, q=15.9, axis=0) + np.percentile(T_ens, q=50, axis=0)),
                              E_array=T.E_array)
T_upper = om.Vector(vector=(np.percentile(T_ens, q=84.1, axis=0) - np.percentile(T_ens, q=50, axis=0)),
                              E_array=T.E_array)

# Convert T to gsf
gsf = om.Vector(E_array=T.E_array, vector=om.div0(T.vector, 2*np.pi*T.E_array**3))
gsf_lower = om.Vector(E_array=T_lower.E_array, vector=om.div0(T_lower.vector, 2*np.pi*T_lower.E_array**3))
gsf_upper = om.Vector(E_array=T_upper.E_array, vector=om.div0(T_upper.vector, 2*np.pi*T_upper.E_array**3))



axgsf_transf.errorbar(gsf.E_array[:-3], gsf.transform(const=B, alpha=alpha).vector[:-3],
                      yerr=gsf_std.transform(const=B, alpha=alpha).vector[:-3],
                      label=r"$\gamma$SF, transformed", linewidth=0.9,
                      fmt="o", markersize=2, capsize=1, color=color)

# Load and add published, normalized data:
gsf_published = np.loadtxt("data/gsf_56Fe_FG05_NaI.txt", skiprows=2)
gsf_published[:, 1] *= 1000
axgsf_transf.errorbar(gsf_published[:, 1], gsf_published[:, 2], yerr=gsf_published[:, 3],
                      fmt="<", markersize=2, color="seagreen", linewidth=0.9,
                      label=r"$\gamma$SF, Larsen $\textit{et\,\,al.}$")

#axgsf_transf.set_title("gsf, transformed")
axgsf_transf.legend()
axgsf_transf.set_yscale("log")

# Axis labels
axrho_transf.set_title("(a) Level density")
axgsf_transf.set_title(r"(b) $\gamma$SF")

axrho_transf.set_ylabel(r"$\rho \,\, (\mathrm{MeV}^{-1})$")
axgsf_transf.set_ylabel(r"$f \,\, (\mathrm{MeV}^{-3})$")

axrho_transf.set_xlabel(r"$E_x (\mathrm{keV})$")
axgsf_transf.set_xlabel(r"$E_\gamma (\mathrm{keV})$")

axrho_transf.set_ylim([5e-4, 1e3])
axgsf_transf.set_ylim([2e-10,1e-6])
axgsf_transf.set_xlim([1600,11500])


f.set_size_inches(4, 6)
plt.tight_layout(True)

f.savefig("Fe56_rho_and_T_fit_alpha_transform-percentile_uncerts.pdf")

plt.show()

In [ ]:
# Plot rho and gsf ensembles

f, (axrho, axgsf) = plt.subplots(2, 1)

opacity = 0.1

# rho
rho_E_array_midbin = rho.E_array + rho.calibration()["a1"]/2
for i_ens in range(len(rho_ens[:, 0])):
    label = "individual members" if i_ens == 0 else None
    axrho.plot(rho_E_array_midbin, rho_ens[i_ens, :]*A*np.exp(alpha*rho_E_array_midbin), ".--",
               linewidth=0.1, alpha=opacity,
               color="fuchsia", label=label)
rho_mean = om.Vector(E_array=rho.E_array, vector=rho_ens.mean(axis=0))
rho_mean.transform(const=A, alpha=alpha).plot(ax=axrho, color="darkblue",
                                              linestyle="--", linewidth=1, label="mean fit")

axrho.legend()
axrho.set_yscale("log")
#axrho.set_xlim([0, 2000])
#axrho.set_ylim([0, 50])
axrho.set_title("(a) Level density")

# gsf

i_gsf_l = 0
i_gsf_h = -3

gsf_E_array_midbin = gsf.E_array + gsf.calibration()["a1"]/2
gsf_ens_transformed = np.zeros(T_ens.shape)
for i_ens in range(len(T_ens[:, 0])):
    label = "individual members" if i_ens == 0 else None
    gsf_ens_transformed[i_ens, :] = T_ens[i_ens, :]*B*np.exp(alpha*gsf_E_array_midbin)/(2*np.pi*gsf_E_array_midbin**3)
    axgsf.plot(gsf_E_array_midbin[i_gsf_l:i_gsf_h], gsf_ens_transformed[i_ens, i_gsf_l:i_gsf_h], ".--", linewidth=0.1,
               alpha=opacity, color="fuchsia", label=label)
gsf_mean = om.Vector(E_array=gsf.E_array,
                     vector=gsf_ens_transformed.mean(axis=0))
axgsf.plot(gsf_mean.E_array[i_gsf_l:i_gsf_h], gsf_mean.vector[i_gsf_l:i_gsf_h], color="darkblue", linewidth=1, linestyle="--", label="mean fit")

axgsf.legend()
axgsf.set_yscale("log")
#axgsf.set_xlim([1000, 4500])
#axgsf.set_ylim([0, 1e-7])
axgsf.set_title(r"(b) $\gamma$SF")


# Axis labels
axrho.set_ylabel(r"$\rho \,\, (\mathrm{MeV}^{-1})$")
axgsf.set_ylabel(r"$f \,\, (\mathrm{MeV}^{-3})$")
axrho.set_xlabel(r"$E_x (\mathrm{keV})$")
axgsf.set_xlabel(r"$E_\gamma (\mathrm{keV})$")


axrho.set_ylim([5e-4, 1e3])
axgsf.set_ylim([2e-10,1e-6])
axgsf_transf.set_xlim([1600,12000])

f.set_size_inches(4, 6)
plt.tight_layout(True)
f.savefig("Fe56_fit_rho_gsf_transformed_with_uncertainty_band.pdf")

plt.show()